# Generador de personas aleatorias

In [3]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import sacar_curp, quitar_acentos
from shapely.geometry import Polygon, Point

In [9]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv')
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for _ in range(n):
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].reset_index()

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'][0], margen_ap['lim_sup.altura.h.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'][0], margen_ap['lim_sup.peso.h.kg'][0]), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'][0], margen_ap['lim_sup.altura.m.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'][0], margen_ap['lim_sup.peso.m.kg'][0]), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()

		ciudad = ct_ciudad['ciudad'].values[0]
		estado = ct_ciudad['estado'].values[0]

		tel = f'({ct_ciudad["clave_lada"].values[0]})' + str(rm.randint(1000000, 9999999))

		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		direccion = fake.street_address()
		direcc_calle = fake.street_name()
		direcc_numero = rm.randint(1, 999)
		direcc_cp = fake.postcode()

		profesion = profesiones.sample().values[0][0] if edad >= 18 else None
		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			nss2 = ('%02d' % rm.randint(0, 99))
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%02d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'
		else:
			nss = None

		"""
		Latitud y longitud de la ciudad
		"""
		# si el catalogo ya tiene coordenadas puestas, tomar esos valores
		if pd.notna(ct_ciudad['latitud'].values[0]) and pd.notna(ct_ciudad['longitud'].values[0]):
			lat = ct_ciudad['latitud'].values[0]
			lon = ct_ciudad['longitud'].values[0]
		else:
			# si no, buscar en internet la latitud y longitud segun la ciudad
			try:
				geolocator = Nominatim(user_agent = 'geoapiExercises')
				location = geolocator.geocode(f'{ciudad}, {estado}, Mexico', language = 'es_MX')
				if location:
					lat = location.latitude
					lon = location.longitude

					# for _ in range(5):
					# 	dec_lat = rm.random() / 100
					# 	dec_lon = rm.random() / 100
					# 	dirr = geolocator.reverse((lat + dec_lat, lon + dec_lon), language = 'es', addressdetails = True)
					# 	print(dirr)

					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudad_index_row = ct_ciudad.index.values[0]
					ct_ciudades.loc[ct_ciudad_index_row, 'latitud'] = lat
					ct_ciudades.loc[ct_ciudad_index_row, 'longitud'] = lon
					ct_ciudades.to_csv(file_ciudades, index = False)
				else:
					raise BaseException(f'(idx {_}) No se encontro latitud ni longitud de {ciudad}, {estado}')
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print('\033[31mException:\033[0m')
				print(be)
				lat = lon = np.NaN

		"""
		Zona horaria
		"""
		if not np.isnan(lat) and not np.isnan(lon):
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			timezone = obj.timezone_at(lat = lat, lng = lon)
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')
		# en Enero no hay DST
		#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
		else:
			timezone = tz_offset = None

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'correo': correo,
			'tel': tel,
			# 'calle': '',
			# 'numero': '',
			# 'cp': '',
			'ciudad': ciudad,
			'estado': estado,
			'lat': lat,
			'lon': lon,
			'tz_offset': tz_offset,
			'zona_horaria': timezone,
			'profesion': profesion,
			'curp': curp,
			'nss': nss,
			# 'color_fav': fake.safe_color_name(),
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Yucatan
Campeche
Yucatan
Guerrero
Veracruz
Nayarit
Sonora
Guanajuato
Baja California Sur
Queretaro


,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,correo,...,ciudad,estado,lat,lon,tz_offset,zona_horaria,profesion,curp,nss,color_fav
0,None,Uriel,Benavídez,Nieto,2008-01-22,H,156.10,58.22,A+,ubn08@example.org,...,Xmatkuil,Yucatan,20.860477,-89.625057,-0600,America/Merida,None,BENU080122HYNNTRD6,None,aqua
1,Dra.,Bianca,Baeza,Palacios,1974-12-24,M,164.70,82.18,A+,quinonesclaudia@example.io,...,Isla Aguada,Campeche,18.786480,-91.491450,-0600,America/Merida,Redactor de publicidad,BAPB741224MCCZLN48,17 08 74 3094 6,silver
2,None,Cecilia,Melgar,Almonte,2013-02-05,M,124.88,30.29,A+,cma2013@example.gob,...,Tixkokob,Yucatan,20.975618,-89.348874,-0600,America/Merida,None,MEAC130205MYNLLCS5,None,green
3,Mtro.,Rogelio,Baeza,Razo,1950-12-05,H,182.76,81.89,A+,luis-miguel39@example.com,...,Coacoyula de Alvarez,Guerrero,18.073560,-99.631079,-0600,America/Mexico_City,Químico analítico,BARR501205HGRZZG55,85 55 50 9267 3,yellow
4,None,Trinidad,Peres,Portillo,1918-07-02,M,167.05,52.28,O+,trinidadperesportillo@example.net,...,Lomas de Arena,Veracruz,20.413700,-97.156850,-0600,America/Mexico_City,Trabajador de las artes comunitarias,PEPT180702MVZRRR97,98 56 18 9809 6,blue
5,Ing.,Mercedes,Saiz,Esquivel,1971-12-29,M,155.06,85.29,B+,mse1971@example.net,...,Nuevo Vallarta,Nayarit,20.702758,-105.288511,-0600,America/Bahia_Banderas,Psicoterapeuta del movimiento de la danza,SAEM711229MNTZSR81,79 52 71 8027 7,silver
6,None,Claudia,Guajardo,Castañeda,2018-10-19,M,100.79,14.21,A+,claudiaguajardocastaneda@example.io,...,Esperanza,Sonora,27.579135,-109.930974,-0700,America/Hermosillo,None,GUCC181019MSRJSLE3,None,gray
7,Dra.,María Elena,Ceja,Longoria,1927-05-13,M,154.58,51.24,A-,mariaelenacejalongoria@example.org,...,Corralejo de Hidalgo,Guanajuato,20.510839,-101.631369,-0600,America/Mexico_City,Trabajador de desarrollo comunitario,CELM270513MGTJNR80,51 31 27 9025 8,blue
8,None,Magdalena,Pantoja,Luna,1957-08-27,M,158.86,51.63,B+,magdalena_pantoja@example.org,...,Buena Vista,Baja California Sur,23.037198,-109.719616,-0700,America/Mazatlan,Tecnólogo Arquitectónico,PALM570827MBSNNG05,70 56 57 9856 5,lime
9,Dr.,Maximiliano,Zedillo,Cabán,1956-03-16,H,183.08,71.33,O+,mzedillo@example.gob,...,Tequisquiapan,Queretaro,20.542216,-99.947899,-0600,America/Mexico_City,Psicologo clínico,ZECM560316HQTDBX73,98 25 56 1215 9,teal


In [ ]:
def generate_random_data(lat, lon, num_rows = 5, save_file_name = None):
	data = []
	for _ in range(num_rows):
		hex1 = '%012x' % rm.randrange(16 ** 12)
		flt = float(rm.randint(0, 100))
		dec_lat = rm.random() / 100
		dec_lon = rm.random() / 100
		data.append((hex1.lower(), flt, lon + dec_lon, lat + dec_lat))
	if save_file_name:
		df = pd.DataFrame(data)
		df.to_csv(save_file_name, header = ['hex1', 'flt', 'dec_lat', 'dec_lon'], index = False)
	return data

def polygon_random_points(poly: list, num_points = 1):
	# https://nominatim.openstreetmap.org/search.php?q=Hermosillo+Sonora+Mexico&polygon_geojson=1&format=json
	# https://medium.com/the-data-journal/a-quick-trick-to-create-random-lat-long-coordinates-in-python-within-a-defined-polygon-e8997f05123a
	# Defining the randomization generator
	poly = Polygon(poly)
	min_x, min_y, max_x, max_y = poly.bounds
	points = []
	while len(points) < num_points:
		random_point = Point([rm.uniform(min_x, max_x), rm.uniform(min_y, max_y)])
		if random_point.within(poly): points.append(random_point)
	return points

In [52]:
# mas info en https://randomuser.me/documentation
url_api = 'https://randomuser.me/api?format=csv'
params = {
	'results': 100,
	#'nat': 'es',
	#'password': '8-16,lower,upper,number,special',
	'exc': 'nat,picture,id,login,registered'
}
query = ''
for k, v in params.items():
	if isinstance(v, int): v = str(v)
	query += '&' + k + '=' + v
url = url_api + query
people = pd.read_csv(url)
people.sample(8)

,gender,name.title,name.first,name.last,location.street.number,location.street.name,location.city,location.state,location.country,location.postcode,location.coordinates.latitude,location.coordinates.longitude,location.timezone.offset,location.timezone.description,email,dob.date,dob.age,phone,cell
71,male,Mr,Christian,Gilbert,7074,Parker Rd,Mackay,Victoria,Australia,6318,-65.3658,-98.5630,+8:00,"Beijing, Perth, Singapore, Hong Kong",christian.gilbert@example.com,1958-01-18T01:59:10.346Z,64,00-2205-7121,0468-031-612
29,female,Ms,Liva,Pedersen,6386,Kildevangen,Assens,Hovedstaden,Denmark,80127,53.8008,-139.2006,+2:00,"Kaliningrad, South Africa",liva.pedersen@example.com,1954-07-23T11:29:28.489Z,68,21992541,15088751
21,female,Miss,Andrea,Jørgensen,6496,Vesterhavsgade,Ugerløse,Danmark,Denmark,53798,15.1058,-51.3289,+5:00,"Ekaterinburg, Islamabad, Karachi, Tashkent",andrea.jorgensen@example.com,1977-02-09T04:00:55.373Z,45,53913262,89410192
14,female,Miss,Geartsje,Keeris,4829,Gloucesterstraat,Mastenbroek,Flevoland,Netherlands,72660,-82.8162,-111.0220,-3:00,"Brazil, Buenos Aires, Georgetown",geartsje.keeris@example.com,1959-01-18T12:02:03.729Z,63,(710)-466-4602,(659)-121-6000
63,male,Mr,Yahya,Tang,1151,Vettaliveien,Øyenkilen,Aust-Agder,Norway,5598,81.7548,-57.5620,+5:30,"Bombay, Calcutta, Madras, New Delhi",yahya.tang@example.com,1965-07-22T20:25:06.118Z,57,79474644,92912583
68,male,Mr,Querubim,da Luz,9738,Rua Dom Pedro Ii,Boa Vista,Mato Grosso do Sul,Brazil,40786,9.4291,16.1268,-7:00,Mountain Time (US & Canada),querubim.daluz@example.com,1960-05-04T03:52:54.142Z,62,(82) 2111-3093,(50) 9939-0517
15,female,Miss,Brooke,Williams,7588,St Aubyn Street,Napier,Southland,New Zealand,96732,54.6276,-131.6838,-1:00,"Azores, Cape Verde Islands",brooke.williams@example.com,1986-10-19T01:43:58.785Z,36,(006)-767-3902,(391)-524-6024
50,male,Mr,Vicente,Soto,284,Avenida de América,Toledo,Región de Murcia,Spain,76114,-48.4810,71.4883,+3:00,"Baghdad, Riyadh, Moscow, St. Petersburg",vicente.soto@example.com,1982-09-11T16:39:42.445Z,40,964-685-721,640-381-041
